In [1]:
import csv
import numpy as np
import collections

In [2]:
data = []
counter = 0
with open('plays_clean2.csv', 'r') as f:
    reader = csv.DictReader(f)
    for i,row in enumerate(reader):
        if not row['off'] and not row['def']:
            print(i, row)
            counter += 1
        data.append(row)
print(counter)

0


In [3]:
grouped_by_game = collections.defaultdict(list)
for line in data:
    grouped_by_game[line['gameid']].append(line)
grouped_by_game = list(grouped_by_game.values())

In [4]:
for game in grouped_by_game:
    for play_dict in game:
        for key in play_dict:
            try:
                play_dict[key] = int(play_dict[key])
            except:
                pass

In [5]:
def put_into_bucket(play_dict):
    yard = play_dict['ydline'] // 10
    down = play_dict['down']
    if play_dict['togo'] <= 2:
        togo = 0
    elif play_dict['togo'] <= 4:
        togo = 1
    elif play_dict['togo'] <= 6:
        togo = 2
    elif play_dict['togo'] <= 8:
        togo = 3
    elif play_dict['togo'] <= 10:
        togo = 4
    elif play_dict['togo'] <= 15:
        togo = 5
    elif play_dict['togo'] <= 20:
        togo = 6
    else:
        togo = 7
    if play_dict['off'] < play_dict['def']:
        posession = 1
    else:
        posession = 0
    return (yard, down, togo, posession)

In [9]:
def group_by_score_change(game, original_scores = None):
    if not game:
        return []
    if original_scores is None:
        original_scores = collections.defaultdict(int)
    changed_team = None
    changed_score = None
    for i, play in enumerate(game):
        if original_scores[play['off']] != play['offscore']:
            changed_team = play['off']
            changed_score = play['offscore'] - original_scores[changed_team]
            break
        if original_scores[play['def']] != play['defscore']:
            changed_team = play['def']
            changed_score = play['defscore'] - original_scores[changed_team]
            break     
    
    if changed_team is None:
        return [('', 0,game)]
    
    new_scores = collections.defaultdict(int)
    new_scores[play['off']] = play['offscore']
    new_scores[play['def']] = play['defscore']
    return [(changed_team, changed_score, game[:i+1])] + group_by_score_change(game[i+1:], new_scores)    

In [10]:
def bucketable(play):
    if not play['down']:
        return False
    if play['down'] < 1:
        return False
    if play['down'] > 4:
        return False
    if not play['togo']:
        return False
    if play['togo'] < 1:
        return False
    if not play['ydline']:
        return False
    if play['ydline'] < 1:
        return False
    if not play['off'] or not play['def']:
        return False
    return True
    


In [34]:
def possession(play):
    if play_dict['off'] < play_dict['def']:
        return 1
    else:
        return 0

def bucket_group(group):
    global test
    test = group
    team, diff, group = group
    answer = []
    teams = set()

    for play in group:
        teams.add(play['off'])
        teams.add(play['def'])
    if '' in teams:
        teams.remove('')
    if team:
        #print(teams)
        pos = 1- sorted(teams).index(team)
    if diff != 0:
        group = group[:-1]
    for i, play in enumerate(group):
        if play['time'] in (1800, 3600):
            if i == 0 or group[i-1]['time'] in (1800, 3600):
                answer.append([6])
        if bucketable(play):
            answer.append(put_into_bucket(play))
        else:
            answer.append([6])
    if team:
        
        if answer and answer[-1] == [6]:
            answer = answer[:-1]
        if diff == 2:
            answer.append([1, pos])
            answer.append([5, pos])
        elif diff == 3:
            answer.append([0, pos])
            answer.append([5, pos])
        elif diff == 6:
            answer.append([2, pos])
            answer.append([5, pos])
        elif diff == 7:
            answer.append([2, pos])
            answer.append([3, pos])
            answer.append([5, pos])
        elif diff == 8:
            answer.append([2, pos])
            answer.append([4, pos])
            answer.append([5, pos])
        elif diff != 0:
            answer.append([6])
    return answer
        

In [12]:
def bucket_game(game):
    grouped = group_by_score_change(game)
    answer = []
    for group in grouped:
        answer += bucket_group(group)
    return answer
        


In [35]:
bucket_sequence = []
for game in grouped_by_game:
    bucket_sequence += bucket_game(game)
    bucket_sequence += [6]

In [36]:
len(bucket_sequence)

492232

In [40]:
import pickle
with open('basic_buckets.pkl', 'wb') as f:
    pickle.dump(bucket_sequence, f, -1)

In [ ]:
len(grouped_by_game)

In [ ]:
for play

In [ ]:
#dump cell

counter = 0
features = ['down', 'togo']
for game in grouped_by_game:
    for i, play in enumerate(game):
        missing = False
        for feature in features:
            if not play[feature]:
                missing = True
        if missing:
            counter += 1
            scores = collections.defaultdict(set)
            num_gaps = 1
            for j in range(i-1, i + 1):
                if 0 <= j < len(game):
                    scores[game[j]['off']].add(game[j]['offscore'])
                    scores[game[j]['def']].add(game[j]['defscore'])
                    for feature in features:
                        if not game[j][feature]:
                            num_gaps += 1
            score_changed = False
            for team in scores:
                if len(scores) >= 1:
                    score_changed = True
                #if max(scores[team]) - min(scores[team]) in (2,3):
                #    num_gaps -= 1
                #if max(scores[team]) - min(scores[team]) in (6,7,8):
                #    num_gaps -= 2
            #if num_gaps > 0:
            #    #print(play)
            #    pass
            if not score_changed:
                print(play)
                
                
for j, game in enumerate(grouped_by_game):
    scores = collections.defaultdict(list)
    for play in game:
        scores[play['off']].append(play['offscore'])
        scores[play['def']].append(play['defscore'])

    for i in range(2, len(game)):
        for team in scores:
            try:
                if len(set((scores[team][i], scores[team][i-1], scores[team][i-2]))) == 3:
                    print(j, i,scores[team][i], scores[team][i-1], scores[team][i-2])
            except:
                pass